In [1]:
import asyncio

async def example_function():
  while True: 
    # print item
    print(f"Runnning")
    # await sleep for one second
    await asyncio.sleep(1)
    # print the next statement
    print(f"Done running")

async def main():
  # begin running task
  task = asyncio.create_task(example_function())
  # momentarily pause to begin the await
  await asyncio.sleep(5)
  # resume function
  task.cancel()

# calls main
await main()

Runnning
Done running
Runnning
Done running
Runnning


CancelledError: 

Done running
Runnning
Done running
Runnning
Done running
Runnning


In [ ]:
import asyncio
import time
import numpy as np

global state 

class State():
    def __init__(self):
        self.state = {}
        self.state_lock = asyncio.Lock() # Global Lock for safe state updates

class Agent():
    def __init__(self, id):
        self.id = id

    async def execute(self):
        """Simulate an agent performing a task."""
        randint = np.random.randint(1, 6)
        factorial = 1
        for i in range(1, randint + 1):
            factorial *= i  # Simulating some computation

        await asyncio.sleep(randint)  # Simulate API call or delay

        return randint

    async def agent_loop(self, id, ostate, start_time, time_limit):
        """Each agent runs its task repeatedly until the time limit is reached."""
        while time.time() - start_time < time_limit:
            result = await self.execute()
            async with ostate.state_lock:
                if id not in ostate.state.keys():
                    ostate.state[id] = []
                ostate.state[id].append(result)
            # print(f"Agent {x} completed task with result: {result}")

async def main(agents, time_limit, state):
    """Start agent loops and run them asynchronously until the time limit."""
    start_time = time.time()
    
    # Create a task for each agent that will keep running
    tasks = [asyncio.create_task(x.agent_loop(x.id, state, start_time, time_limit)) for x in agents]
    
    # Wait for all tasks to stop when time limit is reached
    await asyncio.gather(*tasks)

# Define agents and time limit
ostate = State()
agents = [Agent(0), Agent(1), Agent(2), Agent(3)]
time_limit = 30  # Run for 10 seconds

start_time = time.time()
await main(agents, time_limit, ostate)
end_time = time.time()

print(f"Total execution time: {end_time - start_time:.2f} seconds")
for agent, results in sorted(ostate.state.items()):
    print(f"Agent {agent}: {results}")


Total execution time: 32.07 seconds
Agent 0: [2, 3, 5, 2, 5, 1, 3, 3, 2, 1, 5]
Agent 1: [4, 5, 4, 5, 3, 2, 1, 3, 3]
Agent 2: [1, 4, 4, 5, 1, 2, 2, 3, 4, 5]
Agent 3: [5, 4, 3, 5, 4, 1, 2, 4, 3]


In [1]:
import asyncio
import time
import numpy as np
from openai import OpenAI
from openai import AsyncOpenAI

# Load in API Key and Handle Errors
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = AsyncOpenAI(api_key = openai_api_key)


global state 

class State():
    def __init__(self):
        self.state = {}
        self.state_lock = asyncio.Lock() # Global Lock for safe state updates

class Agent():
    def __init__(self, id):
        self.id = id
        self.messages = []

    async def execute(self):
        # Query the LLM with full stack of current messages
        completion = await client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = self.messages
        )
        return completion.choices[0].message # This returns only the AI's response in text without any "context". 


    async def agent_loop(self, ostate, start_time, time_limit):
        """Each agent runs its task repeatedly until the time limit is reached."""
        while time.time() - start_time < time_limit:
            self.messages.append({'role':'system', 'content':'Return the name of a random capital city of the World'})
            random_wait = .01 * np.random.randint(0, 5)
            await asyncio.sleep(random_wait)
            task = asyncio.create_task(self.execute())
            result = await task
            async with ostate.state_lock:
                if self.id not in ostate.state.keys():
                    ostate.state[self.id] = []
                ostate.state[self.id].append(f"{result.content}, {time.time() - start_time}")
            
            await asyncio.sleep(0.1)
            # print(f"Agent {x} completed task with result: {result}")

async def main(agents, time_limit, state):
    """Start agent loops and run them asynchronously until the time limit."""
    start_time = time.time()
    
    # Create a task for each agent that will keep running
    tasks = [asyncio.create_task(x.agent_loop(state, start_time, time_limit)) for x in agents]
    
    # Wait for all tasks to stop when time limit is reached
    await asyncio.gather(*tasks)

# Define agents and time limit
ostate = State()
agents = [Agent(0), Agent(1), Agent(2), Agent(3)]
time_limit = 15  # Run for 10 seconds

start_time = time.time()
await main(agents, time_limit, ostate)
end_time = time.time()

print(f"Total execution time: {end_time - start_time:.2f} seconds")
for agent, results in sorted(ostate.state.items()):
    print(f"Agent {agent}: {results}")


Total execution time: 15.55 seconds
Agent 0: ['Warsaw, 0.6603217124938965', 'Tokyo, 1.162505865097046', 'Helsinki, 1.7051441669464111', 'Oslo, 2.2559573650360107', 'Berlin, 2.7698464393615723', 'Tokyo, 3.41169810295105', 'Tokyo, 3.929084300994873', 'Nairobi, 4.464965105056763', 'Bishkek, 4.995843410491943', 'Hanoi, 5.583045721054077', 'Helsinki, 6.350046634674072', 'Bucharest, 6.932950735092163', 'Tokyo, 7.481956958770752', 'Tokyo, 8.08025336265564', 'Hanoi, 8.719014406204224', 'Helsinki, 10.483686447143555', 'Vienna, 11.3879976272583', 'Tunis, 11.992833137512207', 'Oslo, 12.629380464553833', 'Bangkok, 13.21861982345581', 'Lisbon, 14.00692892074585', 'Tokyo, 14.612683057785034', 'Lisbon, 15.244638919830322']
Agent 1: ['Nairobi, 0.5871882438659668', 'Lima, 1.189368724822998', 'Lisbon, 1.741905927658081', 'Bamako, 2.2960007190704346', 'Tokyo, 2.892963409423828', 'Helsinki, 3.432673215866089', 'Vienna, 4.433759927749634', 'Tunis, 4.987709045410156', 'Tallinn, 5.56728982925415', 'Hanoi, 6.

In [7]:
import asyncio
import time
import numpy as np
from openai import OpenAI
from openai import AsyncOpenAI

# Load in API Key and Handle Errors
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = AsyncOpenAI(api_key = openai_api_key)


global state 

class State():
    def __init__(self):
        self.state = {}
        self.state_lock = asyncio.Lock() # Global Lock for safe state updates

class Agent():
    def __init__(self, id):
        self.id = id
        self.messages = []

    async def execute(self):
        # Query the LLM with full stack of current messages
        completion = await client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = self.messages
        )
        return completion.choices[0].message # This returns only the AI's response in text without any "context". 


    async def agent_loop(self, ostate, start_time, time_limit):
        """Each agent runs its task repeatedly until the time limit is reached."""
        while time.time() - start_time < time_limit:
            self.messages.append({'role':'system', 'content':'Return the name of a random capital city of the World'})
            random_wait = .01 * np.random.randint(0, 5)
            await asyncio.sleep(random_wait)
            task = asyncio.create_task(self.execute())
            result = await task
            async with ostate.state_lock:
                if self.id not in ostate.state.keys():
                    ostate.state[self.id] = []
                ostate.state[self.id].append(f"{result.content}, {time.time() - start_time}")
            
            await asyncio.sleep(0.1)
            # print(f"Agent {x} completed task with result: {result}")

async def main(agents, time_limit, state):
    """Start agent loops and run them asynchronously until the time limit."""
    start_time = time.time()
    
    # Create a task for each agent that will keep running
    tasks = [asyncio.create_task(x.agent_loop(state, start_time, time_limit)) for x in agents]
    
    # Wait for all tasks to stop when time limit is reached
    await asyncio.gather(*tasks)

# Define agents and time limit
ostate = State()
agent = Agent(0)
agent.messages.append({"role":"user", "content":"What's your favorite color?"})
x = await agent.execute()
print(x)
print(type(x))


ChatCompletionMessage(content='I don’t have personal preferences, but I can help you find information about colors or discuss their meanings and associations! Do you have a favorite color?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>


In [9]:
import openai
def chat_msg():
  return openai.types.chat.chat_completion_message.ChatCompletionMessage({'role':'user', 'content':'Example'})

print(chat_msg())

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [ ]:
openai.types.chat.chat_completion_message.ChatCompletionMessage(content='No response received due to limit error. Please try again.')